# SVD decomposition with `scipy`

Import the required modules

In [2]:
import scipy.linalg as la
import numpy as np

Generate a random 5x4 matrix

In [3]:
np.random.seed(0)

A = np.random.rand(5,4)
A

array([[0.5488135 , 0.71518937, 0.60276338, 0.54488318],
       [0.4236548 , 0.64589411, 0.43758721, 0.891773  ],
       [0.96366276, 0.38344152, 0.79172504, 0.52889492],
       [0.56804456, 0.92559664, 0.07103606, 0.0871293 ],
       [0.0202184 , 0.83261985, 0.77815675, 0.87001215]])

## `<numpy|scipy>.linalg.svd` 

Equivalent implementations:
- https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.svd.html



### Full SVD (`full_matrices = True` - default)

**input:**

$A \in \mathbb{R}^{m\times n}$

**output:**

$U \in \mathbb{R}^{m\times m}, S \in \mathbb{R}^{m\times n}, V \in \mathbb{R}^{n\times n}$

or, more precisely

$\boldsymbol{\sigma} \in \mathbb{R}^{q} $

where $q = \min(m,n)$.


In [4]:
U, s, VT = np.linalg.svd(A)
#U, s, VT = la.svd(A)
print('U shape: ', U.shape)
print('s shape: ', s.shape)
print('VT shape: ', VT.shape)

U shape:  (5, 5)
s shape:  (4,)
VT shape:  (4, 4)


Build the matrix $S$

In [5]:
S = np.zeros(A.shape)
for i in range(len(s)):
    S[i, i] = s[i]
S

array([[2.64618677, 0.        , 0.        , 0.        ],
       [0.        , 0.83351254, 0.        , 0.        ],
       [0.        , 0.        , 0.70753001, 0.        ],
       [0.        , 0.        , 0.        , 0.29842614],
       [0.        , 0.        , 0.        , 0.        ]])

In [6]:
S = la.diagsvd(s, A.shape[0], A.shape[1])
S

array([[2.64618677, 0.        , 0.        , 0.        ],
       [0.        , 0.83351254, 0.        , 0.        ],
       [0.        , 0.        , 0.70753001, 0.        ],
       [0.        , 0.        , 0.        , 0.29842614],
       [0.        , 0.        , 0.        , 0.        ]])

Reconstruct the matrix $A$

In [7]:
A_svd = np.matmul(U, np.matmul(S,VT))
# equivalently: A_svd = U @ S @ VT
print(f"err: {(la.norm(A - A_svd) / la.norm(A))}")

err: 4.624308861561109e-16


### Thin SVD (`full_matrices = False`)

**input:**

$A \in \mathbb{R}^{m\times n}$

**output:**

$U \in \mathbb{R}^{m\times q}, S \in \mathbb{R}^{q\times q}, V \in \mathbb{R}^{n\times q}$

or, more precisely

$\boldsymbol{\sigma} \in \mathbb{R}^{q} $

where $q = \min(m,n)$.



In [8]:
U, s, VT = la.svd(A, full_matrices=False)
print('U shape: ', U.shape)
print('s shape: ', s.shape)
print('VT shape: ', VT.shape)

U shape:  (5, 4)
s shape:  (4,)
VT shape:  (4, 4)


Build the matrix $S$

In [9]:
S = np.diag(s)
S

array([[2.64618677, 0.        , 0.        , 0.        ],
       [0.        , 0.83351254, 0.        , 0.        ],
       [0.        , 0.        , 0.70753001, 0.        ],
       [0.        , 0.        , 0.        , 0.29842614]])

Reconstruct the matrix $A$

In [10]:
A_svd = np.matmul(U, np.matmul(S,VT))
print(f"err: {la.norm(A - A_svd) / la.norm(A)}")

err: 4.624308861561109e-16


### A note on vectorization
Vectorization refers to the practice of replacing explicit loops with high-level mathematical operations that act on entire arrays or matrices at once. This leads to much better performance because it replaces slow Python loops with fast, optimized C and Fortran operations.

Indeed, we could be inclined to reconstruct $A_k$ with a for loop and the explicit formula
$$A_k = \sigma_1 u_1 v_1^T + ... + \sigma_k u_k v_k^T.$$

Let's measure the time taken for this operation for a matrix $A$ that is a bit larger

In [23]:
import time

A = np.random.rand(500, 400)
U, s, VT = la.svd(A, full_matrices=False)
S = np.diag(s)


Time the reconstruction with a for loop

In [24]:
start_time = time.time()

A_reconstructed_loop = np.zeros_like(A)  # Initialize a matrix of zeros
for i in range(len(S)):
    A_reconstructed_loop += s[i] * np.outer(U[:, i], VT[i, :])

loop_time = time.time() - start_time

Time the vectorized reconstruction using matrix multiplication

In [25]:
start_time = time.time()

# here we are using broadcasting to avoid the creation of a diagonal matrix
# see: https://numpy.org/doc/stable/user/basics.broadcasting.html
A_reconstructed_vectorized = (U * s) @ VT

vectorized_time = time.time() - start_time

We compare the results

In [26]:
print(f"Time for reconstruction using for loop: {loop_time:.6f} seconds")
print(f"Time for vectorized reconstruction: {vectorized_time:.6f} seconds")
print(f"Vectorized is {loop_time / vectorized_time:.1f} times faster than the loop")

difference = np.abs(A_reconstructed_loop - A_reconstructed_vectorized).max()
print(f"Difference between the two reconstructions: {difference:.6e}")

Time for reconstruction using for loop: 0.073372 seconds
Time for vectorized reconstruction: 0.001937 seconds
Vectorized is 37.9 times faster than the loop
Difference between the two reconstructions: 2.553513e-15
